## SAVING OUTPUT FILE


### Compressed file - Using pickle format

In [349]:
import pickle

In [350]:
def save_to_file(data_dic, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(data_dic, f)

In [351]:
fs = "hhhh"
v_info = "skjdfld"
kp_vec = [1, 3, 3]
kp_w_scores_vec = [1, 2, 3, 4, 5]
video_time = 10

In [352]:
data = {
    'fs_info': {'fs':fs},
    'v_info': v_info,
    'kp': kp_vec,
    'kp_w_scores_vec': kp_w_scores_vec,
    'video_time': video_time
}

In [353]:
save_to_file(data, 'aquivo')   


In [354]:
infile = open('aquivo','rb')
new_dict = pickle.load(infile)
infile.close()

In [355]:
print(new_dict)

{'fs_info': {'fs': 'hhhh'}, 'v_info': 'skjdfld', 'kp': [1, 3, 3], 'kp_w_scores_vec': [1, 2, 3, 4, 5], 'video_time': 10}


### Uncompressed file standard - Better for real-time processing

In [356]:
import json
import cv2
import numpy as np
import os


import sys

sys.path.append("../utils/")
import drawing


In [357]:
import tensorflow as tf

# Newral Network
interpreter = tf.lite.Interpreter(model_path='../scripts/lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

Necessary Functions

In [1]:
# Constants

# Joints Dictionary
KEYPOINT_DICT = {
    'nose':0,
    'left_eye':1,
    'right_eye':2,
    'left_ear':3,
    'right_ear':4,
    'left_shoulder':5,
    'right_shoulder':6,
    'left_elbow':7,
    'right_elbow':8,
    'left_wrist':9,
    'right_wrist':10,
    'left_hip':11,
    'right_hip':12,
    'left_knee':13,
    'right_knee':14,
    'left_ankle':15,
    'right_ankle':16
} 

# Joint parings 
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [359]:
# Function to get the parings of the desired joints
def getPairings(desired_kp, kp_dict, kp_pairings, neural_network):
    new_kp_dict = {}
    new_kp_pairings = {}

    for kp_name in desired_kp:
        new_kp_dict[kp_name] = kp_dict[kp_name]

    for pos, color in kp_pairings.items():
        p1, p2 = pos
        if (p1 in list(new_kp_dict.values())) and (p2 in list(new_kp_dict.values())):
            p1_tf = list(new_kp_dict.values()).index(p1)
            p2_tf = list(new_kp_dict.values()).index(p2)
            new_kp_pairings[(p1_tf, p2_tf)] = color

    return new_kp_pairings

In [360]:
# Function to select only the desired joints
def selectJoints(image, keypoints, desired_keypoints, kp_dict, neural_network):
    image_height, image_width, _ = image.shape
    print(image.shape)
    print(keypoints)

    selected_joints = np.zeros([len(desired_keypoints), 2])
    selected_joints[:] = np.NaN

    # print("KEYPOINTS | ")
    # if(keypoints != None):
    for i in range(len(desired_keypoints)):
        joint = desired_keypoints[i]
        
        idx = list(kp_dict.keys()).index(joint)
        selected_joints[i, :] = keypoints[idx, :]
 
    return selected_joints

In [361]:
# get the keypoints above the trashhold and change vector to numpy array
def transformDATA(kp_w_scores_vec, confidence_threshold, frame_width, frame_height):
    keypoints_vec = np.zeros([len(list(KEYPOINT_DICT.values())), 2])
    
    y, x, c = frame_width, frame_height, 3
    shaped = np.squeeze(np.multiply(kp_w_scores_vec, [y,x,1]))
    
    j = 0
    for i in range(len(shaped)):
        if i in list(KEYPOINT_DICT.values()):
            ky, kx, kp_conf = shaped[i]
            if kp_conf > confidence_threshold:
                keypoints_vec[j, 0] = kx
                keypoints_vec[j, 1] = ky
            else:
                keypoints_vec[j, 0] = np.nan
                keypoints_vec[j, 1] = np.nan
            j+=1
    return keypoints_vec

In [362]:
# Function to get the absolute path
def getAbsolutePath():
    absolute_main_path = os.path.dirname(os.path.abspath(__file__))
    return absolute_main_path

Functions to generate the output file

In [363]:
# Function to define the file metadata
def setMetadata(video_name, mapping, pairs, video_path, summary="None"):
    if video_name == 0:
        video_path = 0
        
        cap = cv2.VideoCapture(video_path)
        length = 0
        fps = 0
    
    else:
        current_path = os.getcwd()
        video_path = current_path + video_path

        cap = cv2.VideoCapture(video_path)
        print("VIDEO PATH: ", video_path)
        print("CAP: ", cap)

        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))


    _, image = cap.read()
    frame_width = image.shape[0]
    frame_height = image.shape[1]
    
    cap.set(2, 0.0)

    # Metadata dictionary
    file_metadata = {
        'video_name': video_name,
        'n_frames': length,
        'n_points': len(mapping),
        'frame_width': frame_width,
        'frame_height': frame_height,
        'fps': fps,
        'keypoints_names': mapping,
        'keypoints_pairs': pairs,
        'summary': summary
    }
    
    return file_metadata, cap

In [364]:
# Function to write the data to the output file
def writeToDATA(file_path, data, write_mode='w'):
    with open(file_path, write_mode) as f:
        f.write(json.dumps(data))
        f.write('\n')

Code to write and save the output file

In [366]:
# Variables used in this axample
video_path = "/../examples/right-rafilsk1.mp4"
video_out_path = ""
profile = ['right_hip', 'right_knee', 'right_ankle']
file_path = "right"+'.data'

In [368]:
# For the real time loop (example)

file_metadata, cap = setMetadata("RAFA FRONTAL", KEYPOINT_DICT, profile, video_path)
writeToDATA(file_path, file_metadata, write_mode='w+')

# Get video path
current_path = os.getcwd()
video_path = current_path + video_path

cap = cv2.VideoCapture(video_path)

has_frame, image = cap.read()
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = image.shape[0]
frame_height = image.shape[1]
cap.release() 

cap = cv2.VideoCapture(video_path)

# try:
while(True):

    #----------------------------------------------------------------------------------
    ret, frame = cap.read()
    if not ret:
        break
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke() 
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Key points
    keypoints = transformDATA(keypoints_with_scores, 0.3, frame_width, frame_height)

    pose_selected = profile

    keypoint_pairings = getPairings(pose_selected, KEYPOINT_DICT, EDGES, "movenet")
    selected_joints = selectJoints(frame, keypoints, pose_selected, KEYPOINT_DICT, 'movenet')
    
    # Draw the joints and pairings
    drawing.draw_connections(frame, selected_joints, keypoint_pairings)
    drawing.draw_keypoints(frame, selected_joints)
    
    cv2.imshow('MoveNet Lightning', frame)

    # ESC para sair
    k = cv2.waitKey(25) & 0xFF
    if k == 27:
        break
    #----------------------------------------------------------------------------------

        
    file_data = {
        'keypoints': selected_joints.tolist()
    }
    print("FILE DATA: " , file_data)
    print()
    writeToDATA(file_path, file_data, write_mode='a')

cap.release()
cv2.destroyAllWindows()

# except Exception as e:
#     print(e)
#     print("END LOOP")

VIDEO PATH:  c:\Users\rafer\ema_motion_analysis\notebooks/../examples/right-rafilsk1.mp4
CAP:  <VideoCapture 0000018EC2E188D0>
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
(560, 560, 3)
[[309.16742325 174.59737062]
 [308.12674522 167.20372677]
 [308.14840794 166.94978237]
 [289.9911499  163.30046654]
 [292.8821373  163.00411463]
 [269.16463852 191.47045374]
 [280.19496441 190.08891344]
 [251.62952423 232.95526505]
 [280.26692867 254.08510447]
 [281.0851717  293.58705997]
 [288.74662876 293.09502602]
 [268.51178646 302.32434273]
 [278.57470036 303.33815098]
 [265.79184771 413.68366718]
 [268.0213213  413.5306263 ]
 [257.74001122 505.35375595]
 [259.51551914 510.14288425]]
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
(560, 560, 3)
[[309.57130432 173.4292531 ]
 [308.26770306 165.96287727]
 [308.53593349 165.57448387]
 [289.37317848 162.1793294 ]
 [291.11837864 162.21132278]
 [266.67711496 191.15888119]
 [282.31891155 189.82487202]
 [251.24743938 230.230937  ]
 [282.54491806 253.23283195]
 [279.64097977 293.52380